In [135]:
from b2heavy.TwoPointFunctions.utils     import correlation_diagnostics
from b2heavy.TwoPointFunctions.types2pts import CorrelatorIO,Correlator, plot_effective_coeffs
from b2heavy.TwoPointFunctions.fitter    import StagFitter, p_value, standard_p

import numpy             as np
import gvar              as gv
import matplotlib.pyplot as plt
import pandas            as pd
import lsqfit

from tqdm import tqdm

from scipy import linalg as la

import copy
import corrfitter as cf

import h5py

In [136]:
BINSIZE  = {
    'MediumCoarse':13,
    'Coarse-2':    16,
    'Coarse-1':    11,
    'Coarse-Phys': 19,
    'Fine-1':      16,
    'Fine-Phys':   16,
    'SuperFine':   22
}

In [137]:
def Tmax(y,errmax=0.3):
    for i,el in enumerate(y):
        if abs(gv.sdev(el)/gv.mean(el))>errmax:
            return i
    return len(y)

In [138]:
# Correlator specifics
ENSEMBLE = 'Coarse-1'
MESON    = 'Dst'
MOMENTUM = '300'

DATA_DIR = '/Users/pietro/code/data_analysis/BtoD/Alex/'

SMSLIST  = ['1S-1S','d-d','d-1S']

# Specifics of the analysis
TRANGE_EFF = (10,20) 
TRANGE     = (5,20)
NSTATES    = 3
JKFIT      = False

In [139]:
io   = CorrelatorIO(ENSEMBLE,MESON,MOMENTUM,PathToDataDir=DATA_DIR)
stag = StagFitter(
    io       = io,
    jkBin    = BINSIZE[ENSEMBLE],
    smearing = SMSLIST
)
cov_specs = dict(scale=True, shrink=True, cutsvd=1e-12)

In [181]:
nconf = StagFitter(io,jkBin=1).data.shape[2]

In [140]:
effm,effa = stag.meff(TRANGE_EFF,**cov_specs)

/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in sqrt
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in log
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )
/Users/pietro/code/software/B2heavy/b2heavy/TwoPointFunctions/types2pts.py:38: RuntimeWarning: invalid value encountered in arccosh
  m = np.arccosh( (y[(it+1)%len(y)]+y[(it-1)%len(y)])/y[it]/2 )


In [141]:
NEXC = 3
TMIN = 5

In [143]:
xdata,ydata,yjk = stag.format(alljk=True,scale=True,shrink=True,svd=1e-12)

xfit = {}
yfit = {}
yfitjk = {}
for (sm,pl),y in ydata.items():
    xfit[sm,pl] = xdata[TMIN:Tmax(y)]
    yfit[sm,pl] = ydata[sm,pl][TMIN:Tmax(y)]
    yfitjk[sm,pl] = yjk[sm,pl][:,TMIN:Tmax(y)]

yfit_dict   = np.concatenate([yfit[k] for k in stag.keys])
yfitjk_dict = np.hstack([yfitjk[k] for k in stag.keys])

## linear

In [158]:
def fexp(Nt):
    return lambda t,E,Z: Z * ( np.exp(-E*t) + np.exp(-E*(Nt-t)) ) 


def corr2(mes,Nstates,Nt,sm,pol):
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    def aux(t,p):
        c2 = 0.
        for n in range(Nstates):
            Ephy = sum(p[f'dE.{mes}'][:n+1])
            Z0   = p[f'Z.{mes}.{sm1}.{pol}'][n]   * p[f'Z.{mes}.{sm2}.{pol}'][n]
            c2 += fexp(Nt)(t,Ephy,Z0)

            Eosc = sum(p[f'dE.{mes}.o'][:n+1]) 
            Z1   = p[f'Z.{mes}.{sm1}.{pol}.o'][n] * p[f'Z.{mes}.{sm2}.{pol}.o'][n]
            c2 += fexp(Nt)(t,Eosc,Z1) * (-1)**((t+1))

        return c2

    return aux

In [159]:
def fitfcn(xdata,pdict):
    aux = []
    for smr,pol in stag.keys:
        fcn = corr2(stag.info.meson,NEXC,stag.Nt,smr,pol)
        aux.append(fcn(xdata[smr,pol],pdict))
    return np.concatenate(aux)

In [160]:
ampl = {
    '1S': '1(1)',
    'd' : '0.01(50)'
}

pr = {}
for smr,pol in effa:
    s1,s2 = smr.split('-')
    if s1==s2:
        aux = effa[smr,pol] # * 2 * effm

        z0 = f'{aux.mean:.6f}({"1.0" if s1=="1S" else "0.5"})'

        pr[f'Z.{MESON}.{s1}.{pol}']    = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]
        pr[f'Z.{MESON}.{s1}.{pol}'][0] = z0 
        pr[f'Z.{MESON}.{s1}.{pol}.o']  = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]

pr[f'dE.{MESON}']   = ['0.5(5)' for n in range(NEXC)]; pr[f'dE.{MESON}'][0] = f'{effm.mean:.2f}(0.5)'
pr[f'dE.{MESON}.o'] = ['0.5(5)' for n in range(NEXC)]

logprior = gv.BufferDict()
for k,p in pr.items():
    logprior[f'log({k})'] = gv.log(gv.gvar(p))

prior = gv.BufferDict()
for k,p in pr.items():
    prior[k] = gv.gvar(p)


In [161]:
for k,p in prior.items():
    print(k,p)

Z.Dst.1S.Bot [0.8(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Bot.o [1.0(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Par [1.1(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.1S.Par.o [1.0(1.0) 1.0(1.0) 1.0(1.0)]
Z.Dst.d.Bot [0.02(50) 0.01(50) 0.01(50)]
Z.Dst.d.Bot.o [0.01(50) 0.01(50) 0.01(50)]
Z.Dst.d.Par [0.02(50) 0.01(50) 0.01(50)]
Z.Dst.d.Par.o [0.01(50) 0.01(50) 0.01(50)]
dE.Dst [1.20(50) 0.50(50) 0.50(50)]
dE.Dst.o [0.50(50) 0.50(50) 0.50(50)]


In [164]:
fit = lsqfit.nonlinear_fit(
    data  = (xfit,yfit_dict),
    fcn   = fitfcn,
    prior = prior
)
print(fit)

Least Square Fit:
  chi2/dof [dof] = 0.79 [78]    Q = 0.92    logGBF = 1269.3

Parameters:
  Z.Dst.1S.Bot 0     0.896 (53)      [  0.8 (1.0) ]  
               1     -0.82 (25)      [  1.0 (1.0) ]  *
               2     0.933 (81)      [  1.0 (1.0) ]  
Z.Dst.1S.Bot.o 0     0.272 (75)      [  1.0 (1.0) ]  
               1     1.266 (68)      [  1.0 (1.0) ]  
               2      0.30 (26)      [  1.0 (1.0) ]  
  Z.Dst.1S.Par 0     1.059 (61)      [  1.1 (1.0) ]  
               1     -0.77 (26)      [  1.0 (1.0) ]  *
               2     1.089 (92)      [  1.0 (1.0) ]  
Z.Dst.1S.Par.o 0     0.174 (53)      [  1.0 (1.0) ]  
               1     0.983 (69)      [  1.0 (1.0) ]  
               2      0.43 (21)      [  1.0 (1.0) ]  
   Z.Dst.d.Bot 0    0.1417 (66)      [  0.02 (50) ]  
               1    -0.623 (71)      [  0.01 (50) ]  *
               2     0.051 (29)      [  0.01 (50) ]  
 Z.Dst.d.Bot.o 0    0.0280 (75)      [  0.01 (50) ]  
               1     0.148 (17)      [  0.

## reparameterized

In [166]:
def fexp(Nt):
    return lambda t,E,Z: Z * ( np.exp(-E*t) + np.exp(-E*(Nt-t)) ) 


def reparameterized(mes,Nstates,Nt,sm,pol):
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    def aux(t,p):
        c2 = 0.
        for n in range(Nstates):
            Ephy = sum(p[f'dE.{mes}'][:n+1])
            Z0   = p[f'Z.{mes}.{sm1}.{pol}'  ][n]**2 * p[f'Z.{mes}.{sm2}.{pol}'  ][n]**2
            c2 += fexp(Nt)(t,Ephy,Z0)

            Eosc = sum(p[f'dE.{mes}.o'][:n+1]) 
            Z1   = p[f'Z.{mes}.{sm1}.{pol}.o'][n]**2 * p[f'Z.{mes}.{sm2}.{pol}.o'][n]**2
            c2 += fexp(Nt)(t,Eosc,Z1) * (-1)**((t+1))

        return c2

    return aux

def fitfcn(xdata,pdict):
    aux = []
    for smr,pol in stag.keys:
        fcn = reparameterized(stag.info.meson,NEXC,stag.Nt,smr,pol)
        aux.append(fcn(xdata[smr,pol],pdict))
    return np.concatenate(aux)

In [215]:
ampl = {
    '1S': '1(1)',
    'd' : '0.01(50)'
}

pr = {}
for smr,pol in effa:
    s1,s2 = smr.split('-')
    if s1==s2:
        aux = np.sqrt(effa[smr,pol])

        # z0 = f'{aux.mean:.6f}({"1.0" if s1=="1S" else "0.5"})'
        z0 = f'{aux.mean:.6f}(1.0)'

        pr[f'Z.{MESON}.{s1}.{pol}']    = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]
        pr[f'Z.{MESON}.{s1}.{pol}'][0] = z0 
        pr[f'Z.{MESON}.{s1}.{pol}.o']  = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]

pr[f'dE.{MESON}']   = ['0.5(5)' for n in range(NEXC)]; pr[f'dE.{MESON}'][0] = f'{effm.mean:.2f}(0.5)'
pr[f'dE.{MESON}.o'] = ['0.5(5)' for n in range(NEXC)]

In [216]:
prior = gv.BufferDict()
for k,p in pr.items():
    if k.startswith('dE'):
        # prior[f'log({k})'] = gv.log(gv.gvar(p))
        prior[k] = gv.gvar(p)
    else:
        prior[k] = gv.gvar(p)

In [217]:
fit = lsqfit.nonlinear_fit(
    data  = (xfit,yfit_dict),
    fcn   = fitfcn,
    prior = prior,
    verbose = True
)

`ftol` termination condition is satisfied.
Function evaluations 84, initial cost 4.8426e+11, final cost 6.1337e+01, first-order optimality 4.02e-04.


In [218]:
print(fit)

Least Square Fit:
  chi2/dof [dof] = 1.6 [78]    Q = 0.00094    logGBF = 1245.5

Parameters:
  Z.Dst.1S.Bot 0   1.0416 (36)     [  0.9 (1.0) ]  
               1     0.02 (98)     [  1.0 (1.0) ]  
               2     0.70 (12)     [  1.0 (1.0) ]  
Z.Dst.1S.Bot.o 0    0.636 (42)     [  1.0 (1.0) ]  
               1    1.260 (20)     [  1.0 (1.0) ]  
               2     0.16 (89)     [  1.0 (1.0) ]  
  Z.Dst.1S.Par 0   1.1101 (51)     [  1.1 (1.0) ]  
               1    0.624 (32)     [  1.0 (1.0) ]  
               2     0.95 (13)     [  1.0 (1.0) ]  
Z.Dst.1S.Par.o 0    0.541 (30)     [  1.0 (1.0) ]  
               1     0.91 (19)     [  1.0 (1.0) ]  
               2     1.13 (15)     [  1.0 (1.0) ]  
   Z.Dst.d.Bot 0   0.3947 (25)     [  0.1 (1.0) ]  
               1    0.245 (15)     [  0.01 (50) ]  
               2   -1.123 (69)     [  0.01 (50) ]  **
 Z.Dst.d.Bot.o 0    0.204 (15)     [  0.01 (50) ]  
               1   -0.447 (22)     [  0.01 (50) ]  
               2   -0

## reparameterized (`E` also)

In [265]:
def fexp(Nt):
    return lambda t,E,Z: Z * ( np.exp(-E*t) + np.exp(-E*(Nt-t)) ) 

def reparameterizedE(mes,Nstates,Nt,sm,pol):
    sm1,sm2 = sm.split('-')
    mix = sm1!=sm2

    def aux(t,p):
        erg    = np.exp(p[f'dE.{mes}'])
        ergo   = np.exp(p[f'dE.{mes}.o'])
        erg[0] = p[f'dE.{mes}'][0]

        c2 = 0.
        for n in range(Nstates):
            Ephy = sum(p[f'dE.{mes}'][:n+1])
            Z0   = p[f'Z.{mes}.{sm1}.{pol}'  ][n]**2 * p[f'Z.{mes}.{sm2}.{pol}'  ][n]**2
            c2 += fexp(Nt)(t,Ephy,Z0)

            Eosc = sum(p[f'dE.{mes}.o'][:n+1]) 
            Z1   = p[f'Z.{mes}.{sm1}.{pol}.o'][n]**2 * p[f'Z.{mes}.{sm2}.{pol}.o'][n]**2
            c2 += fexp(Nt)(t,Eosc,Z1) * (-1)**((t+1))

        return c2
    
    return aux

In [282]:
# Format and calculate covariance
xdata,ydata,yjk = stag.format(alljk=True,scale=True,shrink=True,svd=1e-12)

# Cut data withing desired fitting range
tmaxs = {}
xfit, yfit, yfitjk = {},{},{}
for sm,pl in stag.keys:
    y = ydata[sm,pl]
    tmax = Tmax(y)

    xfit[sm,pl] = xdata[TMIN:tmax]
    yfit[sm,pl] = ydata[sm,pl][TMIN:tmax]
    yfitjk[sm,pl] = yjk[sm,pl][:,TMIN:tmax]

    tmaxs[sm,pl] = tmax

# Flatten data
yfit_dict   = np.concatenate([yfit[k] for k in stag.keys])
yfitjk_dict = np.hstack([yfitjk[k] for k in stag.keys])

# Define fitting function
def fitfcn(xd,pdict):
    aux = []
    for smr,pol in stag.keys:
        fcn = reparameterizedE(stag.info.meson,NEXC,stag.Nt,smr,pol)
        aux.append(fcn(xd[smr,pol],pdict))
    return np.concatenate(aux)

In [283]:
pr = {}
for smr,pol in effa:
    s1,s2 = smr.split('-')
    if s1==s2:
        aux = np.sqrt(effa[smr,pol])

        z0 = f'{aux.mean:.6f}(1.0)'

        pr[f'Z.{MESON}.{s1}.{pol}']    = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]
        pr[f'Z.{MESON}.{s1}.{pol}'][0] = z0 
        pr[f'Z.{MESON}.{s1}.{pol}.o']  = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]

pr[f'dE.{MESON}']   = ['-0.7(1.0)' for n in range(NEXC)]; pr[f'dE.{MESON}'][0] = f'{effm.mean:.2f}(0.5)'
pr[f'dE.{MESON}.o'] = ['-0.7(1.0)' for n in range(NEXC)]

prior = gv.BufferDict()
for k,p in pr.items():
    prior[k] = gv.gvar(p)

In [284]:
fit = lsqfit.nonlinear_fit(
    data  = (xfit,yfit_dict),
    fcn   = fitfcn,
    prior = prior,
    verbose = True
)

`xtol` termination condition is satisfied.
Function evaluations 64, initial cost 7.3864e+117, final cost 1.1943e+07, first-order optimality 3.26e+18.


/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/scipy/optimize/_lsq/common.py:115: RuntimeWarning: overflow encountered in power
  phi_prime = -np.sum(suf ** 2 / denom**3) / p_norm
/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/scipy/optimize/_lsq/common.py:154: RuntimeWarning: divide by zero encountered in scalar divide
  ratio = phi / phi_prime
/Users/pietro/opt/anaconda3/envs/stat/lib/python3.11/site-packages/lsqfit/__init__.py:776: UserWarning: det(fit.cov) < 0 --- roundoff errors? Try an svd cut.
  warnings.warn('det(fit.cov) < 0 --- roundoff errors? Try an svd cut.')


In [285]:
print(fit)

Least Square Fit:
  chi2/dof [dof] = 3.1e+05 [78]    Q = 0    logGBF = -1.1942e+07

Parameters:
  Z.Dst.1S.Bot 0      1.36701 (41)       [  0.9 (1.0) ]  
               1      0.92123 (43)       [  1.0 (1.0) ]  
               2      0.01265 (21)       [  1.0 (1.0) ]  
Z.Dst.1S.Bot.o 0     0.004079 (27)       [  1.0 (1.0) ]  
               1     -8.78(13)e-07       [  1.0 (1.0) ]  *
               2      6.38(20)e-15       [  1.0 (1.0) ]  
  Z.Dst.1S.Par 0      1.23723 (40)       [  1.1 (1.0) ]  
               1      1.14375 (91)       [  1.0 (1.0) ]  
               2      0.00886 (14)       [  1.0 (1.0) ]  
Z.Dst.1S.Par.o 0     0.004070 (26)       [  1.0 (1.0) ]  
               1      7.80(15)e-07       [  1.0 (1.0) ]  
               2      5.82(27)e-15       [  1.0 (1.0) ]  
   Z.Dst.d.Bot 0       0.6101 (11)       [  0.1 (1.0) ]  
               1      0.53937 (37)       [  0.01 (50) ]  *
               2     0.000157 (12)       [  0.01 (50) ]  
 Z.Dst.d.Bot.o 0    0.0002189 (1

#### jk fit

In [235]:
def firstel(dic):
    return {k: dic[k][0] for k in dic}

In [238]:
cov = gv.evalcov(yfit_dict)

res = []
for ijk in tqdm(range(yfitjk_dict.shape[0])):
    data = (xfit,yfitjk_dict[ijk,:],cov)
    fit = lsqfit.nonlinear_fit(
        data  = data,
        fcn   = fitfcn,
        prior = prior,
    )
    res.append(firstel(fit.pmean))

100%|██████████| 90/90 [00:45<00:00,  1.97it/s]


In [240]:
yfitjk_dict.shape

(90, 78)

# with `corrfitter`

In [55]:
xdata,ydata = stag.format(scale=True,shrink=True)

mes = stag.info.meson

models = []
for smr,pol in ydata:
    sm1,sm2 = smr.split('-')
    fcn = cf.Corr2(
        datatag = (smr,pol),
        tp      = stag.Nt,
        s       = (1.,-1.),
        tdata   = xdata,
        tmin    = TMIN,
        tmax    = Tmax(ydata[smr,pol]),
        dE      = (f'dE.{mes}',f'dE.{mes}.o'),
        a       = (f'Z.{mes}.{sm1}.{pol}',f'Z.{mes}.{sm1}.{pol}.o'),
        b       = (f'Z.{mes}.{sm2}.{pol}',f'Z.{mes}.{sm2}.{pol}.o')
    )

    models.append(fcn)

In [56]:
fitter = cf.CorrFitter(models=models)
cfit = fitter.lsqfit(data=ydata,prior=prior)

In [35]:
print(cfit)

Least Square Fit:
  chi2/dof [dof] = 0.85 [90]    Q = 0.84    logGBF = 1464.1

Parameters:
  Z.Dst.1S.Bot 0     0.914 (34)      [  0.8 (1.0) ]  
               1    -0.983 (49)      [  1.0 (1.0) ]  *
               2      0.78 (11)      [  1.0 (1.0) ]  
        dE.Dst 0    1.2179 (56)      [  1.20 (50) ]  
               1     0.290 (30)      [  0.50 (50) ]  
               2     0.389 (32)      [  0.50 (50) ]  
Z.Dst.1S.Bot.o 0     0.309 (56)      [  1.0 (1.0) ]  
               1     1.164 (31)      [  1.0 (1.0) ]  
               2     0.609 (82)      [  1.0 (1.0) ]  
      dE.Dst.o 0     1.153 (25)      [  0.50 (50) ]  *
               1     0.239 (25)      [  0.50 (50) ]  
               2     0.458 (21)      [  0.50 (50) ]  
  Z.Dst.1S.Par 0     1.095 (38)      [  1.1 (1.0) ]  
               1     1.050 (60)      [  1.0 (1.0) ]  
               2      0.80 (11)      [  1.0 (1.0) ]  
Z.Dst.1S.Par.o 0     0.158 (39)      [  1.0 (1.0) ]  
               1     0.940 (31)      [  1.0

# with `b2heavy`

In [17]:
priors = stag.priors(NEXC,Meff=effm,Aeff=effa)
b2d = stag.fit(NEXC, (4,18), priors=priors,**cov_specs)

In [19]:
print(cfit.p['Z.d.Bot'][0], np.exp(b2d.p['Z.d.Bot'][0]) )
print(cfit.p['Z.1S.Bot'][0], np.exp(b2d.p['Z.1S.Bot'][0]) )

print(cfit.p['Z.d.Par'][0], np.exp(b2d.p['Z.d.Par'][0]) )
print(cfit.p['Z.1S.Par'][0], np.exp(b2d.p['Z.1S.Par'][0]) )

print(cfit.p['dE'][0], b2d.p['E'][0] )

KeyError: 'undefined key: Z.d.Bot'

# add 3 pt functions

In [375]:
from b2heavy.FnalHISQMetadata import params
from b2heavy.TwoPointFunctions.utils import jkCorr

mdata = params(ENSEMBLE)

data = h5py.File(f'/Users/pietro/code/data_analysis/BtoD/Alex/Ensembles/FnalHISQ/a{mdata["naSpc"]}/{mdata["hdf5File2"]}')
data = data['data']

In [376]:
s_he = 'k' + mdata['kBStr']
s_li = 'k' + mdata['kDStr']
s_qq = 'm' + mdata['mlStr']

data3 = {}
for k in data.keys():
    st = k.split('_')
    if len(st)==12:
        src,cur,snk,tt,kh,s1,s2,s3,s4,mq,kl,pmom = st

        B_to_Dst  = kh==s_he and mq==s_qq  and kl==s_li
        collinear = cur[-1]==snk[-1]
        par       = collinear and cur[-1]=='1'

        if B_to_Dst and collinear and pmom=='p300':
            pol = 'Par' if par else 'Bot'
            smr = '1S' if s2=='1S' else 'd'
            data3[f'{src}->{cur}->{snk}.{tt}.{smr}.{pol}'] = jkCorr(data[k][:],bsize=BINSIZE[ENSEMBLE])

In [377]:
for k in data3:
    print(k)

P5->A1->V1.T12.1S.Par
P5->A1->V1.T12.d.Par
P5->A1->V1.T13.1S.Par
P5->A1->V1.T13.d.Par
P5->A2->V2.T12.1S.Bot
P5->A2->V2.T12.d.Bot
P5->A2->V2.T13.1S.Bot
P5->A2->V2.T13.d.Bot
P5->A3->V3.T12.1S.Bot
P5->A3->V3.T12.d.Bot
P5->A3->V3.T13.1S.Bot
P5->A3->V3.T13.d.Bot


In [355]:
model = []
for k in data3:
    corr,tt,smr,pol = k.split('.')
    src,cur,snk     = corr.split('->')

    if corr.endswith('3'):
        continue
    
    if corr.endswith('2'):
        othertag = f'{src}->{cur[0]}3->{snk[0]}3.{tt}.{smr}.{pol}'

    fcn = cf.Corr3(
        datatag   = k,
        T         = int(tt[1:]),
        tmin      = 2,
        dEa       = (f'dE.B'  ,f'dE.B.o'),
        dEb       = (f'dE.Dst',f'dE.Dst.o'),
        # a         = (f'Z.B.1S.{pol}',f'Z.B.1S.{pol}.o'),
        a         = (f'Z.B.1S.Unpol',f'Z.B.1S.Unpol.o'),
        b         = (f'Z.Dst.{smr}.{pol}',f'Z.Dst.{smr}.{pol}.o'),
        sa        = (1,-1),
        sb        = (1,-1),
        # otherdata = othertag if pol=='Bot' else None,
        Vnn='Vnn', Vno='Vno', Von='Von', Voo='Voo',
    )

    model.append(fcn)

In [358]:
prior

BufferDict({'Z.Dst.1S.Bot': array([0.8(1.0), 1.0(1.0), 1.0(1.0)], dtype=object), 'Z.Dst.1S.Bot.o': array([1.0(1.0), 1.0(1.0), 1.0(1.0)], dtype=object), 'Z.Dst.1S.Par': array([1.1(1.0), 1.0(1.0), 1.0(1.0)], dtype=object), 'Z.Dst.1S.Par.o': array([1.0(1.0), 1.0(1.0), 1.0(1.0)], dtype=object), 'Z.Dst.d.Bot': array([0.02(50), 0.01(50), 0.01(50)], dtype=object), 'Z.Dst.d.Bot.o': array([0.01(50), 0.01(50), 0.01(50)], dtype=object), 'Z.Dst.d.Par': array([0.02(50), 0.01(50), 0.01(50)], dtype=object), 'Z.Dst.d.Par.o': array([0.01(50), 0.01(50), 0.01(50)], dtype=object), 'dE.Dst': array([1.20(50), 0.50(50), 0.50(50)], dtype=object), 'dE.Dst.o': array([0.50(50), 0.50(50), 0.50(50)], dtype=object)})

In [378]:
ampl = {
    '1S': '1(1)',
    'd' : '0.01(50)'
}

pr = {}
for smr,pol in effa:
    s1,s2 = smr.split('-')
    if s1==s2:
        aux = effa[smr,pol]

        z0 = f'{aux.mean:.6f}({"1.0" if s1=="1S" else "0.5"})'

        pr[f'Z.{MESON}.{s1}.{pol}']    = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]
        pr[f'Z.{MESON}.{s1}.{pol}'][0] = z0 
        pr[f'Z.{MESON}.{s1}.{pol}.o']  = ['1(1)' if s1=='1S' else '0.01(50)' for n in range(NEXC)]

pr[f'dE.{MESON}']   = ['0.5(5)' for n in range(NEXC)]; pr[f'dE.{MESON}'][0] = f'{effm.mean:.2f}(0.5)'
pr[f'dE.{MESON}.o'] = ['0.5(5)' for n in range(NEXC)]

logprior = gv.BufferDict()
for k,p in pr.items():
    logprior[f'log({k})'] = gv.log(gv.gvar(p))

prior = gv.BufferDict()
for k,p in pr.items():
    prior[k] = gv.gvar(p)